# Overview
This notebook should not be needed in future runs. If I've updated notebook 'A2', then the *ReturnHeader* columns will be parsed there. 

In [4]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [5]:
print(pd.__version__)

0.24.2


In [6]:
#http://pandas.pydata.org/pandas-docs/stable/options.html
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 250)

#### Set working directory

In [7]:
#cd '/Users/gsaxton/Dropbox/990 e-file data'

In [8]:
pwd

'C:\\Users\\Gregory'

In [9]:
cd "C:\\Users\\Gregory\\IRS 990 Control Variables\\"

C:\Users\Gregory\IRS 990 Control Variables


# Read in Concordance File
We are going to read in two codebooks. First, there is the 'concordance' file. Specifically, before re-arranging and renaming variables, we will read in the relevant section from the *master concordance* file, and then use this file to identify the relevant 'compensation' variables. In a following notebook, we will be using the *new_variable_name* field as our variable name.

In [10]:
concordance = pd.read_excel('concordance_VERIFIED.xlsx')
print('# of columns:', len(concordance.columns))
print('# of observations:', len(concordance))
concordance[:2]

# of columns: 15
# of observations: 378


,xpath,project,variable_name_new,# of Characters (newly named),variable name notes,PARSING NOTES,OTHER NOTES,description,location_code,part,data_type_xsd,BINARIZE,MongoDB_Name,sub_key,sub_sub_key
0,/Return/ReturnHeader/TaxPeriodEndDate,NaN,F9_00_HD_TAX_PER_END,NaN,NaN,Will be nested under ReturnHeader,NaN,Tax period end date,HEADER,HD,DateType,NaN,TaxPeriodEndDate,NaN,NaN
1,/Return/ReturnHeader/TaxPeriodEndDt,NaN,F9_00_HD_TAX_PER_END,NaN,NaN,Will be nested under ReturnHeader,NaN,Tax period end date,HEADER,HD,DateType,NaN,TaxPeriodEndDt,NaN,NaN


In [11]:
concordance

,xpath,project,variable_name_new,# of Characters (newly named),variable name notes,PARSING NOTES,OTHER NOTES,description,location_code,part,data_type_xsd,BINARIZE,MongoDB_Name,sub_key,sub_sub_key
0,/Return/ReturnHeader/TaxPeriodEndDate,NaN,F9_00_HD_TAX_PER_END,NaN,NaN,Will be nested under ReturnHeader,NaN,Tax period end date,HEADER,HD,DateType,NaN,TaxPeriodEndDate,NaN,NaN
1,/Return/ReturnHeader/TaxPeriodEndDt,NaN,F9_00_HD_TAX_PER_END,NaN,NaN,Will be nested under ReturnHeader,NaN,Tax period end date,HEADER,HD,DateType,NaN,TaxPeriodEndDt,NaN,NaN
2,/Return/ReturnHeader/TaxYear,NaN,F9_00_HD_TAX_YEAR,NaN,NaN,Will be nested under ReturnHeader,NaN,Tax year,HEADER,HD,YearType,NaN,TaxYear,NaN,NaN
3,/Return/ReturnHeader/TaxYr,NaN,F9_00_HD_TAX_YEAR,NaN,NaN,Will be nested under ReturnHeader,NaN,Tax year,HEADER,HD,YearType,NaN,TaxYr,NaN,NaN
4,/Return/ReturnHeader/BuildTS,NaN,F9_00_HD_BUILD_TIME_STAMP,NaN,NaN,Will be nested under ReturnHeader,NaN,Build time stamp - IRS internal field,HEADER,PART-00,NaN,NaN,BuildTS,NaN,NaN
5,/Return/ReturnHeader/BusinessOfficerGrp/SignatureDt,NaN,F9_00_HD_SIGNING_OFFICER_SIGNTR,NaN,NaN,Will be nested under ReturnHeader,NaN,Date of Signing Officer's Signature,HEADER,PART-00,DateType,NaN,BusinessOfficerGrp,SignatureDt,NaN
6,/Return/ReturnHeader/Officer/DateSigned,NaN,F9_00_HD_SIGNING_OFFICER_SIGNTR,NaN,NaN,Will be nested under ReturnHeader,NaN,Date of Signing Officer's Signature,HEADER,PART-00,DateType,NaN,Officer,DateSigned,NaN
7,/Return/ReturnHeader/ReturnTs,NaN,F9_00_HD_TIME_STAMP,NaN,NaN,Will be nested under ReturnHeader,NaN,The date and time when the return was created,HEADER,PART-00,TimestampType,NaN,ReturnTs,NaN,NaN
8,/Return/ReturnHeader/Timestamp,NaN,F9_00_HD_TIME_STAMP,NaN,NaN,Will be nested under ReturnHeader,NaN,The date and time when the return was created,HEADER,PART-00,TimestampType,NaN,Timestamp,NaN,NaN
9,/Return/ReturnHeader/Filer/USAddress/State,NaN,F9_00_HD_FILER_STATE_US,NaN,NaN,Will be nested under ReturnHeader,NaN,Address of Filing Organization (US State),HEADER,PART-00,StateType,NaN,Filer,USAddress,State


In [12]:
concordance = concordance[concordance['xpath'].str.contains('ReturnHeader')]
print('# of columns:', len(concordance.columns))
print('# of observations:', len(concordance))
concordance[:1]

# of columns: 15
# of observations: 11


,xpath,project,variable_name_new,# of Characters (newly named),variable name notes,PARSING NOTES,OTHER NOTES,description,location_code,part,data_type_xsd,BINARIZE,MongoDB_Name,sub_key,sub_sub_key
0,/Return/ReturnHeader/TaxPeriodEndDate,NaN,F9_00_HD_TAX_PER_END,NaN,NaN,Will be nested under ReturnHeader,NaN,Tax period end date,HEADER,HD,DateType,NaN,TaxPeriodEndDate,NaN,NaN


# Read 990 DB into PANDAS 
- In previous round there were 1,547,828 observations; now there are 1,727,056 observations
- I also switched to using the *.pkl* file

In [13]:
df = pd.read_pickle('all filings feb. 2020 - select ReturnHeader variables.pkl')
print('# of columns:', len(df.columns))
print('# of observations:', len(df))
df[:2]

# of columns: 15
# of observations: 1727056


,DLN,EIN,OrganizationName,TaxPeriod,URL,Timestamp,TaxPeriodEndDate,Filer,Officer,TaxYear,BuildTS,ReturnTs,TaxPeriodEndDt,BusinessOfficerGrp,TaxYr
0,93493313013011,232705170,RONALD MCDONALD HOUSE CHARITIES- PHILADELPHIA REGION INC,201012,https://s3.amazonaws.com/irs-form-990/201113139349301301_public.xml,2011-11-09T06:41:09-06:00,2010-12-31,"{'EIN': '232705170', 'Name': {'BusinessNameLine1': 'RONALD MCDONALD HOUSE CHARITIES-', 'BusinessNameLine2': 'PHILADELPHIA REGION INC'}, 'NameControl': 'RONA', 'Phone': '8565826843', 'USAddress': {'AddressLine1': '1525 VALLEY CENTER PARKWAY NO 300...","{'Name': 'ROBERT TRAA', 'Title': 'TREASURER', 'Phone': '8565826843', 'DateSigned': '2011-11-04', 'AuthorizeThirdParty': '1'}",2010,2016-02-24 21:20:13Z,NaN,NaN,NaN,NaN
1,93493313013111,581805618,TORRINGTON VOA ELDERLY HOUSING INC BELL PARK TOWER,201106,https://s3.amazonaws.com/irs-form-990/201113139349301311_public.xml,2011-11-09T07:32:06-08:00,2011-06-30,"{'EIN': '581805618', 'Name': {'BusinessNameLine1': 'TORRINGTON VOA ELDERLY HOUSING INC', 'BusinessNameLine2': 'BELL PARK TOWER'}, 'NameControl': 'TORR', 'Phone': '7033415000', 'USAddress': {'AddressLine1': '1660 DUKE STREET', 'City': 'ALEXANDRIA'...","{'Name': 'THOMAS D TURNBULL', 'Title': 'ASST. SEC/TREAS', 'DateSigned': '2011-11-09'}",2010,2016-02-24 21:20:13Z,NaN,NaN,NaN,NaN


### Collapse

In [30]:
def agg_funcs(x):
    names = {
        #'name': x['variable_name_new'].head(1).values[0],
        'original_names':  list(set(x['MongoDB_Name'].tolist())),
        'data_type_xsd': x['data_type_xsd'].head(1).values[0]
        }
    #THE FOLLOWING SHORTCUT WORKS BUT CHANGES THE ORDER OF THE COLUMNS
    #return pd.Series(names, index = list(names.keys()))
    return pd.Series(names, index=['original_names', 'data_type_xsd'])
new_variables_df = concordance[:].groupby(['variable_name_new']).apply(agg_funcs)
new_variables_df = new_variables_df.reset_index()
print('# of variables:', len(new_variables_df))
new_variables_df[:5]

# of variables: 6


,variable_name_new,original_names,data_type_xsd
0,F9_00_HD_BUILD_TIME_STAMP,[BuildTS],NaN
1,F9_00_HD_FILER_STATE_US,[Filer],StateType
2,F9_00_HD_SIGNING_OFFICER_SIGNTR,"[BusinessOfficerGrp, Officer]",DateType
3,F9_00_HD_TAX_PER_END,"[TaxPeriodEndDt, TaxPeriodEndDate]",DateType
4,F9_00_HD_TAX_YEAR,"[TaxYr, TaxYear]",YearType


In [31]:
new_variables_df['len'] = new_variables_df['original_names'].apply(lambda x: len(x))
print(new_variables_df['len'].value_counts(), '\n')
new_variables_df

2    4
1    2
Name: len, dtype: int64 



,variable_name_new,original_names,data_type_xsd,len
0,F9_00_HD_BUILD_TIME_STAMP,[BuildTS],NaN,1
1,F9_00_HD_FILER_STATE_US,[Filer],StateType,1
2,F9_00_HD_SIGNING_OFFICER_SIGNTR,"[BusinessOfficerGrp, Officer]",DateType,2
3,F9_00_HD_TAX_PER_END,"[TaxPeriodEndDt, TaxPeriodEndDate]",DateType,2
4,F9_00_HD_TAX_YEAR,"[TaxYr, TaxYear]",YearType,2
5,F9_00_HD_TIME_STAMP,"[ReturnTs, Timestamp]",TimestampType,2


### Handle variables with only 1 original name
NOTE:
- Per *IRS 990 e-File Data -- Control Variables (4) -- Fees-for-Services Variables  - Extract from MongoDB and Process -- Part I (Python 3.6).ipynb*, it looks like there is no *FeesForServicesProfFundraisingGrp*
    - Instead, as seen in the concordance file, *FeesForServicesProfFundraising* has both a 'Total' and a 'TotalAmt' key, which suggests this is the only key that did not change names

In [32]:
new_variables_df[new_variables_df['len']!=2]

,variable_name_new,original_names,data_type_xsd,len
0,F9_00_HD_BUILD_TIME_STAMP,[BuildTS],NaN,1
1,F9_00_HD_FILER_STATE_US,[Filer],StateType,1


#### Rename *F9_00_HD_BUILD_TIME_STAMP*

In [33]:
df.rename(columns = {'BuildTS':'F9_00_HD_BUILD_TIME_STAMP'}, inplace = True)
df['F9_00_HD_BUILD_TIME_STAMP'].describe()

count                  1727056
unique                      12
top       2017-02-10 21:41:12Z
freq                    304553
Name: F9_00_HD_BUILD_TIME_STAMP, dtype: object

In [34]:
df['F9_00_HD_BUILD_TIME_STAMP'].value_counts().head()

2017-02-10 21:41:12Z    304553
2016-02-24 21:20:13Z    299019
2019-02-21 02:37:17Z    230149
2015-11-30 17:44:51Z    199014
2016-02-25 16:41:14Z    170329
Name: F9_00_HD_BUILD_TIME_STAMP, dtype: int64

In [20]:
#df = df.drop('fiscal_year', 1)

<br>Drop the two variables from *new_variables_df*

In [35]:
#print(len(new_variables_df))
#new_variables_df = new_variables_df[~new_variables_df['variable_name_new'].isin(['F9_00_HD_TAX_PER_END', 'F9_00_HD_TAX_YEAR'])]
print(len(new_variables_df))

6


##### Save DF

In [31]:
#import timeit
#start_time = timeit.default_timer()
#df.to_pickle('all filings - with 185 newly named control variables.pkl')
#elapsed = timeit.default_timer() - start_time
#print('# of minutes: ', elapsed/60) 

# of minutes:  3.988038138333339


# Combine all columns where *len*==2

### Define Function to combine columns
In Python we can thus create a series of functions that can be used as shortcuts. First we'll create a function called 'combine' that will combine two variables. It takes as *inputs* four things: our dataset/dataframe (*df*), the name we'd like for our new variable (*newvar*), the name of the first variable to combine (*var1*), and the name of the second variable to combine (*var2*).

In [36]:
def combine(df, newvar, var1, var2):
    df[newvar] = np.where(df[var1].notnull(), df[var1], df[var2])
    #print(df[newvar].value_counts().head(), '\n')
    #print('# of missing observations:', len(df[df[newvar].isnull()]))
    #print('# of valid observations:', len(df[df[newvar].notnull()]), '\n')  
    #return df.sample(5)[[newvar, var1, var2, 'DLN']] 
    #print(df[[newvar, var1, var2, 'ObjectId']][:5], '\n\n\n')

#### Do initial check to ensure that no row has values in both columns

In [37]:
for index, row in new_variables_df[new_variables_df['len']==2][:].iterrows():
    #print(row['variable_name_new'])
    print(row['variable_name_new'], row['original_names'][0], row['original_names'][1])
    print('\t\t', len(df[df[row['original_names'][0]].notnull()]))
    print('\t\t', len(df[df[row['original_names'][1]].notnull()]))
    #print(len(df[(df[row['original_names'][0]].isnull()) & (df[row['original_names'][1]].isnull())]), '\n\n')      
    print('OK IF ZERO:', len(df[(df[row['original_names'][0]].notnull()) & (df[row['original_names'][1]].notnull())]), '\n\n')

F9_00_HD_SIGNING_OFFICER_SIGNTR BusinessOfficerGrp Officer
		 1231533
		 495523
OK IF ZERO: 0 


F9_00_HD_TAX_PER_END TaxPeriodEndDt TaxPeriodEndDate
		 1231533
		 495523
OK IF ZERO: 0 


F9_00_HD_TAX_YEAR TaxYr TaxYear
		 1231533
		 495523
OK IF ZERO: 0 


F9_00_HD_TIME_STAMP ReturnTs Timestamp
		 1231533
		 495523
OK IF ZERO: 0 




### Combine

In [38]:
combo_fails = []
for index, row in new_variables_df[new_variables_df['len']==2][:].iterrows():
    print(row['variable_name_new'], row['original_names'][0], row['original_names'][1])
    try:
        combine(df, row['variable_name_new'], row['original_names'][0], row['original_names'][1])
    except:
        print('\n\n\n\n\n***********issue with variable: ', row['variable_name_new'])
        combo_fails.append(row['variable_name_new'])

print(combo_fails)

F9_00_HD_SIGNING_OFFICER_SIGNTR BusinessOfficerGrp Officer
F9_00_HD_TAX_PER_END TaxPeriodEndDt TaxPeriodEndDate
F9_00_HD_TAX_YEAR TaxYr TaxYear
F9_00_HD_TIME_STAMP ReturnTs Timestamp
[]


In [39]:
df.sample(5)

,DLN,EIN,OrganizationName,TaxPeriod,URL,Timestamp,TaxPeriodEndDate,Filer,Officer,TaxYear,F9_00_HD_BUILD_TIME_STAMP,ReturnTs,TaxPeriodEndDt,BusinessOfficerGrp,TaxYr,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_TIME_STAMP,F9_00_HD_SIGNING_OFFICER_SIGNTR__SAFE,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR
4283,93493050006434,941550842,STANFORD SETTLEMENT INC,201306,https://s3.amazonaws.com/irs-form-990/201430509349300643_public.xml,2014-02-19T17:31:19-06:00,2013-06-30,"{'EIN': '941550842', 'Name': {'BusinessNameLine1': 'STANFORD SETTLEMENT INC'}, 'NameControl': 'STAN', 'Phone': '9169271303', 'USAddress': {'AddressLine1': '450 WEST EL CAMINO AVE', 'City': 'SACRAMENTO', 'State': 'CA', 'ZIPCode': '958332299'}}","{'Name': 'SISTER JEANNE FELION', 'Title': 'EXECUTIVE DIRECTOR', 'Phone': '9169271303', 'DateSigned': '2013-11-30', 'AuthorizeThirdParty': 'true'}",2012,2015-11-30 17:44:51Z,NaN,NaN,NaN,NaN,"{'Name': 'SISTER JEANNE FELION', 'Title': 'EXECUTIVE DIRECTOR', 'Phone': '9169271303', 'DateSigned': '2013-11-30', 'AuthorizeThirdParty': 'true'}",2014-02-19T17:31:19-06:00,"{'Name': 'SISTER JEANNE FELION', 'Title': 'EXECUTIVE DIRECTOR', 'Phone': '9169271303', 'DateSigned': '2013-11-30', 'AuthorizeThirdParty': 'true'}",2013-06-30,2012
6994,93493233004339,383414580,GRAND RAPIDS RED PROJECT,201812,https://s3.amazonaws.com/irs-form-990/201932339349300433_public.xml,NaN,NaN,"{'EIN': '383414580', 'BusinessName': {'BusinessNameLine1Txt': 'The Grand Rapids Red Project'}, 'BusinessNameControlTxt': 'GRAN', 'USAddress': {'AddressLine1Txt': '401 Hall Street SE', 'CityNm': 'Grand Rapids', 'StateAbbreviationCd': 'MI', 'ZIPCd'...",NaN,NaN,2019-02-21 02:37:17Z,2019-08-20T15:50:26-04:00,2018-12-31,"{'PersonNm': 'Stephen Alsum', 'PersonTitleTxt': 'Executive Direc', 'PhoneNum': '6164569063', 'SignatureDt': '2019-08-20', 'DiscussWithPaidPreparerInd': 'true'}",2018,"{'PersonNm': 'Stephen Alsum', 'PersonTitleTxt': 'Executive Direc', 'PhoneNum': '6164569063', 'SignatureDt': '2019-08-20', 'DiscussWithPaidPreparerInd': 'true'}",2019-08-20T15:50:26-04:00,"{'PersonNm': 'Stephen Alsum', 'PersonTitleTxt': 'Executive Direc', 'PhoneNum': '6164569063', 'SignatureDt': '2019-08-20', 'DiscussWithPaidPreparerInd': 'true'}",2018-12-31,2018
7809,93493250004347,920177969,COOK INLET LENDING CENTER,201612,https://s3.amazonaws.com/irs-form-990/201742509349300434_public.xml,NaN,NaN,"{'EIN': '920177969', 'BusinessName': {'BusinessNameLine1Txt': 'COOK INLET LENDING CENTER'}, 'BusinessNameControlTxt': 'COOK', 'PhoneNum': '9077933000', 'USAddress': {'AddressLine1Txt': '3510 SPENARD ROAD NO 201', 'CityNm': 'ANCHORAGE', 'StateAbbr...",NaN,NaN,2017-02-10 21:41:12Z,2017-09-07T12:54:29-05:00,2016-12-31,"{'PersonNm': 'JOE DUBLER', 'PersonTitleTxt': 'VP OF FINANCE', 'PhoneNum': '9077933000', 'SignatureDt': '2017-07-26', 'DiscussWithPaidPreparerInd': '1'}",2016,"{'PersonNm': 'JOE DUBLER', 'PersonTitleTxt': 'VP OF FINANCE', 'PhoneNum': '9077933000', 'SignatureDt': '2017-07-26', 'DiscussWithPaidPreparerInd': '1'}",2017-09-07T12:54:29-05:00,"{'PersonNm': 'JOE DUBLER', 'PersonTitleTxt': 'VP OF FINANCE', 'PhoneNum': '9077933000', 'SignatureDt': '2017-07-26', 'DiscussWithPaidPreparerInd': '1'}",2016-12-31,2016
1284,93493240005365,264338279,PROJECT SOLE,201412,https://s3.amazonaws.com/irs-form-990/201512409349300536_public.xml,NaN,NaN,"{'EIN': '264338279', 'BusinessName': {'BusinessNameLine1Txt': 'PROJECT SOLE'}, 'BusinessNameControlTxt': 'PROJ', 'PhoneNum': '2148462042', 'USAddress': {'AddressLine1Txt': '1705 KELLY BLVD', 'CityNm': 'CARROLLTON', 'StateAbbreviationCd': 'TX', 'Z...",NaN,NaN,2016-03-07 17:11:31Z,2015-08-28T17:04:18-05:00,2014-12-31,"{'PersonNm': 'ZACH STEVENS', 'PersonTitleTxt': 'PRESIDENT', 'PhoneNum': '2148462042', 'SignatureDt': '2015-08-28', 'DiscussWithPaidPreparerInd': 'true'}",2014,"{'PersonNm': 'ZACH STEVENS', 'PersonTitleTxt': 'PRESIDENT', 'PhoneNum': '2148462042', 'SignatureDt': '2015-08-28', 'DiscussWithPaidPreparerInd': 'true'}",2015-08-28T17:04:18-05:00,"{'PersonNm': 'ZACH STEVENS'

#### Save DF

In [26]:
import timeit
start_time = timeit.default_timer()
df.to_pickle('all filings feb. 2020 - select ReturnHeader variables.pkl')
elapsed = timeit.default_timer() - start_time
print('# of minutes: ', elapsed/60) 

# of minutes:  0.3671854616666678


### *BusinessOfficerGrp*

In [25]:
def func(x, key1, key2):
    if pd.isnull(x):
        return np.nan
    #else: 
    #    mydict = ast.literal_eval(x)
    elif key1 in x.keys():
        return x[key1]
    elif key2 in x.keys():
        return x[key2]
    else:
        return np.nan

In [40]:
concordance[concordance['sub_key'].notnull()]

,xpath,project,variable_name_new,# of Characters (newly named),variable name notes,PARSING NOTES,OTHER NOTES,description,location_code,part,data_type_xsd,BINARIZE,MongoDB_Name,sub_key,sub_sub_key
5,/Return/ReturnHeader/BusinessOfficerGrp/SignatureDt,NaN,F9_00_HD_SIGNING_OFFICER_SIGNTR,NaN,NaN,Will be nested under ReturnHeader,NaN,Date of Signing Officer's Signature,HEADER,PART-00,DateType,NaN,BusinessOfficerGrp,SignatureDt,NaN
6,/Return/ReturnHeader/Officer/DateSigned,NaN,F9_00_HD_SIGNING_OFFICER_SIGNTR,NaN,NaN,Will be nested under ReturnHeader,NaN,Date of Signing Officer's Signature,HEADER,PART-00,DateType,NaN,Officer,DateSigned,NaN
9,/Return/ReturnHeader/Filer/USAddress/State,NaN,F9_00_HD_FILER_STATE_US,NaN,NaN,Will be nested under ReturnHeader,NaN,Address of Filing Organization (US State),HEADER,PART-00,StateType,NaN,Filer,USAddress,State
10,/Return/ReturnHeader/Filer/USAddress/StateAbbreviationCd,NaN,F9_00_HD_FILER_STATE_US,NaN,NaN,Will be nested under ReturnHeader,NaN,Address of Filing Organization (US State),HEADER,PART-00,StateType,NaN,Filer,USAddress,StateAbbreviationCd


In [41]:
df[:1]

,DLN,EIN,OrganizationName,TaxPeriod,URL,Timestamp,TaxPeriodEndDate,Filer,Officer,TaxYear,F9_00_HD_BUILD_TIME_STAMP,ReturnTs,TaxPeriodEndDt,BusinessOfficerGrp,TaxYr,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_TIME_STAMP,F9_00_HD_SIGNING_OFFICER_SIGNTR__SAFE,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR
0,93493313013011,232705170,RONALD MCDONALD HOUSE CHARITIES- PHILADELPHIA REGION INC,201012,https://s3.amazonaws.com/irs-form-990/201113139349301301_public.xml,2011-11-09T06:41:09-06:00,2010-12-31,"{'EIN': '232705170', 'Name': {'BusinessNameLine1': 'RONALD MCDONALD HOUSE CHARITIES-', 'BusinessNameLine2': 'PHILADELPHIA REGION INC'}, 'NameControl': 'RONA', 'Phone': '8565826843', 'USAddress': {'AddressLine1': '1525 VALLEY CENTER PARKWAY NO 300...","{'Name': 'ROBERT TRAA', 'Title': 'TREASURER', 'Phone': '8565826843', 'DateSigned': '2011-11-04', 'AuthorizeThirdParty': '1'}",2010,2016-02-24 21:20:13Z,NaN,NaN,NaN,NaN,"{'Name': 'ROBERT TRAA', 'Title': 'TREASURER', 'Phone': '8565826843', 'DateSigned': '2011-11-04', 'AuthorizeThirdParty': '1'}",2011-11-09T06:41:09-06:00,"{'Name': 'ROBERT TRAA', 'Title': 'TREASURER', 'Phone': '8565826843', 'DateSigned': '2011-11-04', 'AuthorizeThirdParty': '1'}",2010-12-31,2010


In [28]:
df['F9_00_HD_SIGNING_OFFICER_SIGNTR__SAFE'] = df['F9_00_HD_SIGNING_OFFICER_SIGNTR']

In [71]:
import timeit
start_time = timeit.default_timer()
df['F9_00_HD_SIGNING_OFFICER_SIGNTR'] = df['F9_00_HD_SIGNING_OFFICER_SIGNTR'][:].apply(func, key1='SignatureDt', key2='DateSigned')
#df['F9_09_PC_FEES_FOR_SVCE_MGMT_TOT2'] = df['ReturnHeader'].astype('float')
elapsed = timeit.default_timer() - start_time
print('# of minutes: ', elapsed/60) 

# of minutes:  0.18038617166666882


In [42]:
df[['EIN', 'F9_00_HD_SIGNING_OFFICER_SIGNTR__SAFE', 'F9_00_HD_SIGNING_OFFICER_SIGNTR']].sample(5)

,EIN,F9_00_HD_SIGNING_OFFICER_SIGNTR__SAFE,F9_00_HD_SIGNING_OFFICER_SIGNTR
6162,570115330,"{'PersonNm': 'PAMELA L CHRISTOPHER', 'PersonTitleTxt': 'PRESIDENT & CEO', 'PhoneNum': '8642263454', 'SignatureDt': '2017-11-07', 'DiscussWithPaidPreparerInd': '1'}","{'PersonNm': 'PAMELA L CHRISTOPHER', 'PersonTitleTxt': 'PRESIDENT & CEO', 'PhoneNum': '8642263454', 'SignatureDt': '2017-11-07', 'DiscussWithPaidPreparerInd': '1'}"
5852,840504202,"{'PersonNm': 'Debbie Bresina', 'PersonTitleTxt': 'President', 'PhoneNum': '3034251606', 'SignatureDt': '2017-11-28', 'DiscussWithPaidPreparerInd': '1'}","{'PersonNm': 'Debbie Bresina', 'PersonTitleTxt': 'President', 'PhoneNum': '3034251606', 'SignatureDt': '2017-11-28', 'DiscussWithPaidPreparerInd': '1'}"
3297,481055202,"{'PersonNm': 'JEANNIE BLANCK-GRIFFITH', 'PersonTitleTxt': 'PRESIDENT', 'PhoneNum': '7852724735', 'SignatureDt': '2014-07-11', 'DiscussWithPaidPreparerInd': '1'}","{'PersonNm': 'JEANNIE BLANCK-GRIFFITH', 'PersonTitleTxt': 'PRESIDENT', 'PhoneNum': '7852724735', 'SignatureDt': '2014-07-11', 'DiscussWithPaidPreparerInd': '1'}"
8410,760351808,"{'PersonNm': 'JOHN BURNS', 'PersonTitleTxt': 'PRESIDENT', 'PhoneNum': '7134616533', 'SignatureDt': '2015-04-30', 'DiscussWithPaidPreparerInd': 'true'}","{'PersonNm': 'JOHN BURNS', 'PersonTitleTxt': 'PRESIDENT', 'PhoneNum': '7134616533', 'SignatureDt': '2015-04-30', 'DiscussWithPaidPreparerInd': 'true'}"
9081,741541774,"{'Name': 'ROBERT LUCIO', 'Title': 'EXECUTIVE DIRECTOR', 'Phone': '8303034376', 'DateSigned': '2012-08-08', 'AuthorizeThirdParty': '1'}","{'Name': 'ROBERT LUCIO', 'Title': 'EXECUTIVE DIRECTOR', 'Phone': '8303034376', 'DateSigned': '2012-08-08', 'AuthorizeThirdParty': '1'}"


### *F9_00_HD_FILER_STATE_US*

In [44]:
df[['Filer']].sample(2)

,Filer
3927,"{'EIN': '560623935', 'BusinessName': {'BusinessNameLine1Txt': 'CHARLOTTE COUNTRY DAY SCHOOL'}, 'BusinessNameControlTxt': 'CHAR', 'PhoneNum': '7049434500', 'USAddress': {'AddressLine1Txt': '1440 CARMEL ROAD', 'CityNm': 'CHARLOTTE', 'StateAbbreviat..."
8632,"{'EIN': '042978701', 'BusinessName': {'BusinessNameLine1Txt': 'MEDFIELD COALITION FOR PUBLIC EDUCATION', 'BusinessNameLine2Txt': 'INC'}, 'BusinessNameControlTxt': 'MEDF', 'PhoneNum': '5083591986', 'USAddress': {'AddressLine1Txt': 'C/O ADRIENNE BR..."


In [47]:
def func_onekey(x, key1):
    if pd.isnull(x):
        return np.nan
    #else: 
    #    mydict = ast.literal_eval(x)
    elif key1 in x.keys():
        return x[key1]
    else:
        return np.nan

In [49]:
import timeit
start_time = timeit.default_timer()
df['USAddress'] = df['Filer'][:].apply(func_onekey, key1='USAddress')
#df['F9_09_PC_FEES_FOR_SVCE_MGMT_TOT2'] = df['ReturnHeader'].astype('float')
elapsed = timeit.default_timer() - start_time
print('# of minutes: ', elapsed/60) 

# of minutes:  0.19576037666666554


In [50]:
df[['EIN', 'Filer', 'USAddress']].sample(5)

,EIN,Filer,USAddress
9633,621512543,"{'EIN': '621512543', 'Name': {'BusinessNameLine1': 'SOUTH MEMPHIS ALLIANCE INCORPORATED'}, 'NameControl': 'SOUT', 'Phone': '9017749586', 'USAddress': {'AddressLine1': '1048 S BELLEVUE BLVD', 'City': 'MEMPHIS', 'State': 'TN', 'ZIPCode': '38106'}}","{'AddressLine1': '1048 S BELLEVUE BLVD', 'City': 'MEMPHIS', 'State': 'TN', 'ZIPCode': '38106'}"
8403,203970774,"{'EIN': '203970774', 'BusinessName': {'BusinessNameLine1': 'JERRY AND PAULA BAKER FOUNDATION', 'BusinessNameLine2': 'INC'}, 'BusinessNameControlTxt': 'JERR', 'PhoneNum': '2099627916', 'USAddress': {'AddressLine1': '12930 BOITANO COURT', 'City': '...","{'AddressLine1': '12930 BOITANO COURT', 'City': 'GROVELAND', 'State': 'CA', 'ZIPCode': '95321'}"
5970,382320469,"{'EIN': '382320469', 'BusinessName': {'BusinessNameLine1Txt': 'MICHIGAN SCIENCE TEACHERS', 'BusinessNameLine2Txt': 'ASSOCIATION'}, 'BusinessNameControlTxt': 'MICH', 'PhoneNum': '7349730433', 'USAddress': {'AddressLine1Txt': '1390 EISENHOWER PLACE...","{'AddressLine1Txt': '1390 EISENHOWER PLACE', 'CityNm': 'ANN ARBOR', 'StateAbbreviationCd': 'MI', 'ZIPCd': '48108'}"
4811,953457097,"{'EIN': '953457097', 'BusinessName': {'BusinessNameLine1Txt': 'ST NICHOLAS CEDARS MANOR INC', 'BusinessNameLine2Txt': 'C/O SK MANAGEMENT COMPANY LLC'}, 'BusinessNameControlTxt': 'STNI', 'PhoneNum': '3239302300', 'USAddress': {'AddressLine1Txt': '...","{'AddressLine1Txt': '15910 VENTURA BLVD NO 1400', 'CityNm': 'ENCINO', 'StateAbbreviationCd': 'CA', 'ZIPCd': '91436'}"
8169,202743344,"{'EIN': '202743344', 'BusinessName': {'BusinessNameLine1': 'FOUNDATION FOR AFFORDABLE RENTAL HOUSING'}, 'BusinessNameControlTxt': 'FOUN', 'PhoneNum': '7704331711', 'USAddress': {'AddressLine1': '3330 CUMBERLAND BLVD SUITE 900', 'City': 'ATLANTA',...","{'AddressLine1': '3330 CUMBERLAND BLVD SUITE 900', 'City': 'ATLANTA', 'State': 'GA', 'ZIPCode': '30339'}"


In [51]:
import timeit
start_time = timeit.default_timer()
df['F9_00_HD_FILER_STATE_US'] = df['USAddress'][:].apply(func, key1='State', key2='StateAbbreviationCd')
elapsed = timeit.default_timer() - start_time
print('# of minutes: ', elapsed/60) 

# of minutes:  0.1970622583333314


In [52]:
df[['EIN', 'Filer', 'USAddress', 'F9_00_HD_FILER_STATE_US']].sample(5)

,EIN,Filer,USAddress,F9_00_HD_FILER_STATE_US
2946,710820020,"{'EIN': '710820020', 'BusinessName': {'BusinessNameLine1Txt': 'JOYCORPS PROJECT'}, 'BusinessNameControlTxt': 'JOYC', 'PhoneNum': '4794426211', 'USAddress': {'AddressLine1Txt': 'PO BOX 8696', 'CityNm': 'FAYETTEVILLE', 'StateAbbreviationCd': 'AR', ...","{'AddressLine1Txt': 'PO BOX 8696', 'CityNm': 'FAYETTEVILLE', 'StateAbbreviationCd': 'AR', 'ZIPCd': '72703'}",AR
6507,956095644,"{'EIN': '956095644', 'Name': {'BusinessNameLine1': 'YOUTH TENNIS SAN DIEGO'}, 'NameControl': 'YOUT', 'Phone': '6192219000', 'USAddress': {'AddressLine1': '4490 WEST POINT LOMA BLVD', 'City': 'SAN DIEGO', 'State': 'CA', 'ZIPCode': '92107'}}","{'AddressLine1': '4490 WEST POINT LOMA BLVD', 'City': 'SAN DIEGO', 'State': 'CA', 'ZIPCode': '92107'}",CA
2809,132701075,"{'EIN': '132701075', 'BusinessName': {'BusinessNameLine1Txt': 'DENTON GREEN HOUSING COMPANY INC'}, 'BusinessNameControlTxt': 'DENT', 'USAddress': {'AddressLine1Txt': '500 DENTON AVENUE', 'CityNm': 'GARDEN CITY PARK', 'StateAbbreviationCd': 'NY', ...","{'AddressLine1Txt': '500 DENTON AVENUE', 'CityNm': 'GARDEN CITY PARK', 'StateAbbreviationCd': 'NY', 'ZIPCd': '11040'}",NY
6180,237409712,"{'EIN': '237409712', 'Name': {'BusinessNameLine1': 'THE SONGWRITERS GUILD FOUNDATION'}, 'NameControl': 'SONG', 'Phone': '6157429945', 'USAddress': {'AddressLine1': '5120 VIRGINIA WAY', 'City': 'BRENTWOOD', 'State': 'TN', 'ZIPCode': '37027'}}","{'AddressLine1': '5120 VIRGINIA WAY', 'City': 'BRENTWOOD', 'State': 'TN', 'ZIPCode': '37027'}",TN
4880,233011941,"{'EIN': '233011941', 'Name': {'BusinessNameLine1': 'THE CENTER FOR LOSS AND BEREAVEMENT'}, 'NameControl': 'CENT', 'Phone': '6102224110', 'USAddress': {'AddressLine1': '3847 Skippack Pike', 'AddressLine2': 'PO Box 1299', 'City': 'Skippack', 'State...","{'AddressLine1': '3847 Skippack Pike', 'AddressLine2': 'PO Box 1299', 'City': 'Skippack', 'State': 'PA', 'ZIPCode': '19474'}",PA


#### Save DF

In [53]:
import timeit
start_time = timeit.default_timer()
df.to_pickle('all filings feb. 2020 - select ReturnHeader variables.pkl')
elapsed = timeit.default_timer() - start_time
print('# of minutes: ', elapsed/60) 

# of minutes:  0.7587211866666659


### Check that total number of values in new variable equal sum of prior 2 variables

In [55]:
for index, row in new_variables_df[new_variables_df['len']==2][:].iterrows():
    #print(row['variable_name_new'])
    print(row['variable_name_new'], row['original_names'][0], row['original_names'][1])
    print(len(df[df[row['original_names'][0]].notnull()]) + len(df[df[row['original_names'][1]].notnull()]))    
    print(len(df[df[row['variable_name_new']].notnull()]), '\n')
    #print(len(df[(df[row['original_names'][0]].notnull()) & (df[row['original_names'][1]].notnull())]), '\n')     

F9_00_HD_SIGNING_OFFICER_SIGNTR BusinessOfficerGrp Officer
1727056
1727056 

F9_00_HD_TAX_PER_END TaxPeriodEndDt TaxPeriodEndDate
1727056
1727056 

F9_00_HD_TAX_YEAR TaxYr TaxYear
1727056
1727056 

F9_00_HD_TIME_STAMP ReturnTs Timestamp
1727056
1727056 



<br><br>
From the above we are fine with deleting the 138 variables related to the 69 above variables in *variable_name_new*

### Inspect the Combined and Original Variables
Here I'm showing one variable

In [56]:
df[['F9_00_HD_TAX_YEAR', 'TaxYr', 'TaxYear']].sample(5)

,F9_00_HD_TAX_YEAR,TaxYr,TaxYear
5404,2017,2017,NaN
6923,2015,2015,NaN
7807,2017,2017,NaN
1830,2011,NaN,2011
8602,2010,NaN,2010


### Drop variables

In [57]:
new_variables_df['original_names'].tolist()

[['BuildTS'],
 ['Filer'],
 ['BusinessOfficerGrp', 'Officer'],
 ['TaxPeriodEndDt', 'TaxPeriodEndDate'],
 ['TaxYr', 'TaxYear'],
 ['ReturnTs', 'Timestamp']]

In [58]:
new_variables_df[new_variables_df['len']!=2]['original_names'].tolist()

[['BuildTS'], ['Filer']]

In [59]:
new_variables_df[new_variables_df['len']==2]['original_names'].tolist()

[['BusinessOfficerGrp', 'Officer'],
 ['TaxPeriodEndDt', 'TaxPeriodEndDate'],
 ['TaxYr', 'TaxYear'],
 ['ReturnTs', 'Timestamp']]

In [60]:
flat_list = [item for sublist in new_variables_df[new_variables_df['len']==2]['original_names'].tolist() for item in sublist]
print(len(flat_list))
print(flat_list[:])

8
['BusinessOfficerGrp', 'Officer', 'TaxPeriodEndDt', 'TaxPeriodEndDate', 'TaxYr', 'TaxYear', 'ReturnTs', 'Timestamp']


<br> Flatten a list of lists: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists

In [61]:
print(len([c for c in df.columns.tolist() if c not in flat_list]))
print([c for c in df.columns.tolist() if c not in flat_list])

14
['DLN', 'EIN', 'OrganizationName', 'TaxPeriod', 'URL', 'Filer', 'F9_00_HD_BUILD_TIME_STAMP', 'F9_00_HD_SIGNING_OFFICER_SIGNTR', 'F9_00_HD_TIME_STAMP', 'F9_00_HD_SIGNING_OFFICER_SIGNTR__SAFE', 'F9_00_HD_TAX_PER_END', 'F9_00_HD_TAX_YEAR', 'USAddress', 'F9_00_HD_FILER_STATE_US']


In [62]:
print(len(new_variables_df['variable_name_new'].tolist()))

6


In [63]:
set([c for c in df.columns.tolist() if c not in flat_list]) - set(new_variables_df['variable_name_new'].tolist())

{'DLN',
 'EIN',
 'F9_00_HD_SIGNING_OFFICER_SIGNTR__SAFE',
 'Filer',
 'OrganizationName',
 'TaxPeriod',
 'URL',
 'USAddress'}

<br>The following block drops 310 columns

In [65]:
print(len(df.columns))
df = df[[c for c in df.columns.tolist() if c not in flat_list]]
print(len(df.columns))
df[:1]

14
14


,DLN,EIN,OrganizationName,TaxPeriod,URL,Filer,F9_00_HD_BUILD_TIME_STAMP,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_TIME_STAMP,F9_00_HD_SIGNING_OFFICER_SIGNTR__SAFE,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR,USAddress,F9_00_HD_FILER_STATE_US
0,93493313013011,232705170,RONALD MCDONALD HOUSE CHARITIES- PHILADELPHIA REGION INC,201012,https://s3.amazonaws.com/irs-form-990/201113139349301301_public.xml,"{'EIN': '232705170', 'Name': {'BusinessNameLine1': 'RONALD MCDONALD HOUSE CHARITIES-', 'BusinessNameLine2': 'PHILADELPHIA REGION INC'}, 'NameControl': 'RONA', 'Phone': '8565826843', 'USAddress': {'AddressLine1': '1525 VALLEY CENTER PARKWAY NO 300...",2016-02-24 21:20:13Z,"{'Name': 'ROBERT TRAA', 'Title': 'TREASURER', 'Phone': '8565826843', 'DateSigned': '2011-11-04', 'AuthorizeThirdParty': '1'}",2011-11-09T06:41:09-06:00,"{'Name': 'ROBERT TRAA', 'Title': 'TREASURER', 'Phone': '8565826843', 'DateSigned': '2011-11-04', 'AuthorizeThirdParty': '1'}",2010-12-31,2010,"{'AddressLine1': '1525 VALLEY CENTER PARKWAY NO 300', 'City': 'BETHLEHEM', 'State': 'PA', 'ZIPCode': '18017'}",PA


In [66]:
df = df.drop('F9_00_HD_SIGNING_OFFICER_SIGNTR__SAFE', 1)
df = df.drop('Filer', 1)
df = df.drop('USAddress', 1)

##### Verify

In [67]:
print(len(df.columns.tolist()))

11


In [68]:
set(df.columns.tolist()) - set(new_variables_df['variable_name_new'].tolist())

{'DLN', 'EIN', 'OrganizationName', 'TaxPeriod', 'URL'}

In [69]:
set(new_variables_df['variable_name_new'].tolist()) - set(df.columns.tolist())

set()

In [72]:
df.sample(5)

,DLN,EIN,OrganizationName,TaxPeriod,URL,F9_00_HD_BUILD_TIME_STAMP,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_TIME_STAMP,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR,F9_00_HD_FILER_STATE_US
8828,93493209009012,382872584,SUBSTANCE ABUSE PREVENTION SERVICES,201109,https://s3.amazonaws.com/irs-form-990/201212099349300901_public.xml,2016-02-24 21:20:13Z,2012-07-23,2012-07-27T15:58:56-05:00,2011-09-30,2010,MI
7095,93493139007046,010793507,SOUTH CAROLINA COMMUNITY LOAN FUND,201512,https://s3.amazonaws.com/irs-form-990/201641399349300704_public.xml,2016-09-27 15:27:22Z,2016-05-16,2016-05-16T00:00:00-05:00,2015-12-31,2015,SC
3757,93493135047543,232455915,ACADEMIC ASSETS INC,201206,https://s3.amazonaws.com/irs-form-990/201341359349304754_public.xml,2016-02-24 21:20:13Z,2013-05-15,2013-05-15T11:55:10-05:00,2012-06-30,2011,PA
4462,93493193011596,061427077,YOUNG MARINERS FOUNDATION INC,201508,https://s3.amazonaws.com/irs-form-990/201641939349301159_public.xml,2016-09-27 15:27:22Z,2016-07-11,2016-07-11T15:42:04-05:00,2015-08-31,2014,CT
9085,93493113006016,252120984,FEDERATION OF PITTSBURGH DIOCESAN TEACHERS LOCAL 2400,201512,https://s3.amazonaws.com/irs-form-990/201611139349300601_public.xml,2016-09-27 15:27:22Z,2016-04-22,2016-04-22T08:06:31-05:00,2015-12-31,2015,PA


##### Save DF

In [73]:
import timeit
start_time = timeit.default_timer()
df.to_pickle('all filings feb. 2020 - select ReturnHeader variables.pkl')
elapsed = timeit.default_timer() - start_time
print('# of minutes: ', elapsed/60) 

# of minutes:  0.19093227166666413
